# Agent state occupancy & trajectories

## Init

In [1]:
from pathlib import Path
import numpy as np

In [2]:
save_path = Path("..") / "save"
save_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save')

In [3]:
sessions_path = save_path / "2025-01-19_10-39-52_EastWest_scratch"
sessions_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save/2025-01-19_10-39-52_EastWest_scratch')

In [4]:
data_path = sessions_path / "data.npz"
data_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save/2025-01-19_10-39-52_EastWest_scratch/data.npz')

In [5]:
data_path.exists()

True

In [14]:
data = np.load(data_path, allow_pickle=True)

In [15]:
[item for item in data.keys()]

['rewards',
 'steps',
 'episodes',
 'all_states',
 'all_actions',
 'losses',
 'p',
 'epsilons',
 'weights',
 'biases',
 'weights_val_stats',
 'biases_val_stats',
 'weights_grad_stats',
 'biases_grad_stats',
 'net',
 'env']

## State occupancy

In [16]:
data["all_states"].shape

(20550, 19)

In [17]:
data["all_actions"].shape

(20550,)

In [18]:
data["episodes"]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [19]:
data["rewards"].shape

(100, 3)

In [20]:
data["rewards"].shape

(100, 3)

In [84]:
len(data["losses"][0])

1671

## Trajectories